<img src="https://github.com/hernancontigiani/ceia_memorias_especializacion/raw/master/Figures/logoFIUBA.jpg" width="500" align="center">


# Procesamiento de lenguaje natural
## Custom embedddings con Gensim



### Objetivo
El objetivo es utilizar documentos / corpus para crear embeddings de palabras basado en ese contexto. Se utilizará canciones de bandas para generar los embeddings, es decir, que los vectores tendrán la forma en función de como esa banda haya utilizado las palabras en sus canciones.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import multiprocessing
from gensim.models import Word2Vec

### Datos
Utilizaremos como dataset noticias

In [ ]:
# Descargar la carpeta de dataset
from google.colab import drive
drive.mount('/content/drive')
df= pd.read_excel('/content/drive/MyDrive/Colab Notebooks/df_noticias.xlsx')


#quitar límite de columnas:
pd.options.display.max_columns = None

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
df.head()

,ArticleId,Texto,Categoría
0,1833,worldcom ex-jefe de lanza de la defensa de los...,negocio
1,154,alemán de confianza empresarial diapositivas a...,negocio
2,1101,bbc encuesta indica pesimismo económico de los...,negocio
3,1976,estilo de vida que rige la elección de disposi...,tecnologia
4,917,enron jefes en fue de $168 millones de pago de...,negocio


In [ ]:
df.shape

(1580, 3)

In [ ]:
print("Cantidad de documentos:", df.shape[0])

Cantidad de documentos: 1580


In [ ]:
text = df[["Texto"]]
text.head(10)

,Texto
0,worldcom ex-jefe de lanza de la defensa de los...
1,alemán de confianza empresarial diapositivas a...
2,bbc encuesta indica pesimismo económico de los...
3,estilo de vida que rige la elección de disposi...
4,enron jefes en fue de $168 millones de pago de...
5,howard truanted jugar al billar líder conserva...
6,gales silencio en el grand slam de hablar rhys...
7,francés honor para el director parker director...
8,el gigante del automóvil golpeado por mercedes...
9,fockers de combustible festiva de cine gráfico...


### 1 - Preprocesamiento

In [ ]:
from keras.preprocessing.text import text_to_word_sequence

sentence_tokens = []
# Recorrer todas las filas y transformar las oraciones
# en una secuencia de palabras (esto podría realizarse con NLTK o spaCy también)
for _, row in text[:None].iterrows():
    sentence_tokens.append(text_to_word_sequence(row[0]))

In [ ]:
# Demos un vistazo
sentence_tokens[:2]

[['worldcom',
  'ex',
  'jefe',
  'de',
  'lanza',
  'de',
  'la',
  'defensa',
  'de',
  'los',
  'abogados',
  'de',
  'la',
  'defensa',
  'de',
  'la',
  'ex',
  'worldcom',
  'jefe',
  'de',
  'bernie',
  'ebbers',
  'en',
  'contra',
  'de',
  'una',
  'batería',
  'de',
  'cargos',
  'de',
  'fraude',
  'han',
  'llamado',
  'de',
  'una',
  'empresa',
  'denunciante',
  'como',
  'su',
  'primer',
  'testigo',
  '\xa0cynthia',
  'cooper',
  'worldcom',
  's',
  'ex',
  'jefe',
  'de',
  'contabilidad',
  'interna',
  'que',
  'alertó',
  'a',
  'los',
  'directores',
  'irregulares',
  'prácticas',
  'contables',
  'en',
  'el',
  'gigante',
  'de',
  'las',
  'telecomunicaciones',
  'en',
  'el',
  'año',
  '2002',
  '\xa0sus',
  'advertencias',
  'llevó',
  'al',
  'colapso',
  'de',
  'la',
  'empresa',
  'tras',
  'el',
  'descubrimiento',
  'de',
  'un',
  '11',
  'mil',
  'millones',
  'â£5',
  '7',
  'bn',
  'fraude',
  'contable',
  '\xa0mr',
  'ebbers',
  'se',
  'ha',

### 2 - Crear los vectores (word2vec)

In [ ]:
from gensim.models.callbacks import CallbackAny2Vec
# Durante el entrenamiento gensim por defecto no informa el "loss" en cada época
# Sobrecargamos el callback para poder tener esta información
class callback(CallbackAny2Vec):
    """
    Callback to print loss after each epoch
    """
    def __init__(self):
        self.epoch = 0

    def on_epoch_end(self, model):
        loss = model.get_latest_training_loss()
        if self.epoch == 0:
            print('Loss after epoch {}: {}'.format(self.epoch, loss))
        else:
            print('Loss after epoch {}: {}'.format(self.epoch, loss- self.loss_previous_step))
        self.epoch += 1
        self.loss_previous_step = loss

In [ ]:
# Crearmos el modelo generador de vectores
# En este caso utilizaremos la estructura modelo Skipgram
w2v_model = Word2Vec(min_count=5,    # frecuencia mínima de palabra para incluirla en el vocabulario
                     window=2,       # cant de palabras antes y desp de la predicha
                     vector_size=300,       # dimensionalidad de los vectores
                     negative=20,    # cantidad de negative samples... 0 es no se usa
                     workers=1,      # si tienen más cores pueden cambiar este valor
                     sg=1)           # modelo 0:CBOW  1:skipgram

In [ ]:
# Obtener el vocabulario con los tokens
w2v_model.build_vocab(sentence_tokens)

In [ ]:
# Cantidad de filas/docs encontradas en el corpus
print("Cantidad de docs en el corpus:", w2v_model.corpus_count)

Cantidad de docs en el corpus: 1580


In [ ]:
# Cantidad de words encontradas en el corpus
print("Cantidad de words distintas en el corpus:", len(w2v_model.wv.index_to_key))

Cantidad de words distintas en el corpus: 10180


### 3 - Entrenar embeddings

In [ ]:
# Entrenamos el modelo generador de vectores
# Utilizamos nuestro callback
w2v_model.train(sentence_tokens,
                 total_examples=w2v_model.corpus_count,
                 epochs=20,
                 compute_loss = True,
                 callbacks=[callback()]
                 )

Loss after epoch 0: 5292685.5
Loss after epoch 1: 3920430.5
Loss after epoch 2: 3724520.0
Loss after epoch 3: 3632239.0
Loss after epoch 4: 3471709.0
Loss after epoch 5: 3377674.0
Loss after epoch 6: 3299428.0
Loss after epoch 7: 3235754.0
Loss after epoch 8: 3167882.0
Loss after epoch 9: 3153566.0
Loss after epoch 10: 3070876.0
Loss after epoch 11: 2985612.0
Loss after epoch 12: 2923620.0
Loss after epoch 13: 2860500.0
Loss after epoch 14: 2803740.0
Loss after epoch 15: 2756044.0
Loss after epoch 16: 2718148.0
Loss after epoch 17: 2680824.0
Loss after epoch 18: 2655252.0
Loss after epoch 19: 2632052.0


(9457353, 14325400)

### 4 - Ensayar

In [ ]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["amenaza"], topn=10)

[('denegada', 0.4824664890766144),
 ('interrupción', 0.48106294870376587),
 ('humana', 0.4722875952720642),
 ('inusualmente', 0.46804189682006836),
 ('solidez', 0.46803009510040283),
 ('escala', 0.4626197814941406),
 ('confidencialidad', 0.458187073469162),
 ('energética', 0.45599067211151123),
 ('\xa0existe', 0.4557032287120819),
 ('sustancia', 0.45465993881225586)]

In [ ]:
# Palabras que MENOS se relacionan con...:
w2v_model.wv.most_similar(negative=["problemas"], topn=10)

[('incluye', 0.03214702382683754),
 ('aunque', 0.006601432804018259),
 ('será', 0.006444491446018219),
 ('lectura', 0.0030478197149932384),
 ('les', 0.001626501209102571),
 ('tomó', -0.0005222600302658975),
 ('debut', -0.0035497655626386404),
 ('sería', -0.010465883649885654),
 ('\xa0este', -0.014852160587906837),
 ('escuchar', -0.01786554418504238)]

In [ ]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["banco"], topn=10)

[('\xa0corea', 0.4512261748313904),
 ('sindicato', 0.4447088837623596),
 ('provincia', 0.43892693519592285),
 ('bce', 0.431254118680954),
 ('gobernador', 0.42811280488967896),
 ('sudeste', 0.42377153038978577),
 ('boro', 0.4222494065761566),
 ('\xa0rosneft', 0.41970354318618774),
 ('prestamista', 0.41711604595184326),
 ('océano', 0.41284728050231934)]

In [ ]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["alemania"], topn=5)

[('finlandia', 0.5601865649223328),
 ('suecia', 0.5312897562980652),
 ('bulgaria', 0.5297452807426453),
 ('polonia', 0.5201624035835266),
 ('samoa', 0.5158166885375977)]

In [ ]:
# Ensayar con una palabra que no está en el vocabulario:
w2v_model.wv.most_similar(negative=["died"])

KeyError: ignored

In [ ]:
# el método `get_vector` permite obtener los vectores:
vector_alemania = w2v_model.wv.get_vector("alemania")
print(vector_alemania)

[ 1.75537661e-01 -1.26551881e-01 -5.23785055e-02  5.59446812e-02
  3.98724735e-01 -4.33837771e-01  2.32815713e-01  5.99350095e-01
 -4.00635630e-01  7.97337592e-02  5.23075283e-01 -1.51218608e-01
  4.67543960e-01  4.42970954e-02  1.22020215e-01 -2.37603307e-01
  2.27295578e-01 -2.58809596e-01 -4.22298640e-01  2.18974918e-01
  1.32904097e-01  1.08672000e-01  3.77819568e-01 -3.21580917e-01
  1.59304105e-02  3.37370127e-01 -8.62048268e-02 -2.12130144e-01
 -1.49913579e-01  1.32016227e-01 -1.23214342e-01 -4.37481888e-02
  8.45412984e-02  4.87023890e-02 -2.34394759e-01 -2.26649001e-01
  2.81859506e-02 -1.70486167e-01 -2.39329025e-01  3.14366490e-01
 -2.17717752e-01 -2.32891023e-01  2.48035435e-02  1.36757240e-01
  5.59975542e-02 -2.84564257e-01  2.54916161e-01  5.09349763e-01
  2.28621989e-01 -4.05881368e-02  1.53918564e-01  1.25490557e-02
 -2.38644689e-01  4.61436622e-02 -7.00367808e-01  6.78467512e-01
 -1.35168597e-01  1.20566696e-01  7.56765902e-02  4.41492528e-01
 -2.55660355e-01  1.53474

In [ ]:
# el método `most_similar` también permite comparar a partir de vectores
w2v_model.wv.most_similar(vector_alemania)

[('alemania', 1.0),
 ('finlandia', 0.5601865649223328),
 ('suecia', 0.5312897562980652),
 ('bulgaria', 0.5297452807426453),
 ('polonia', 0.5201624035835266),
 ('samoa', 0.5158166885375977),
 ('chipre', 0.5133373737335205),
 ('islámica', 0.5125570297241211),
 ('república', 0.5070661902427673),
 ('deslice', 0.5054185390472412)]

In [ ]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["alemania"], topn=10)

[('finlandia', 0.5601865649223328),
 ('suecia', 0.5312897562980652),
 ('bulgaria', 0.5297452807426453),
 ('polonia', 0.5201624035835266),
 ('samoa', 0.5158166885375977),
 ('chipre', 0.5133373737335205),
 ('islámica', 0.5125570297241211),
 ('república', 0.5070661902427673),
 ('deslice', 0.5054185390472412),
 ('guillermo', 0.5028282403945923)]

### 5 - Visualizar agrupación de vectores

In [ ]:
from sklearn.decomposition import IncrementalPCA
from sklearn.manifold import TSNE
import numpy as np

def reduce_dimensions(model, num_dimensions = 2 ):

    vectors = np.asarray(model.wv.vectors)
    labels = np.asarray(model.wv.index_to_key)

    tsne = TSNE(n_components=num_dimensions, random_state=0)
    vectors = tsne.fit_transform(vectors)

    return vectors, labels

In [ ]:
# Graficar los embedddings en 2D
import plotly.graph_objects as go
import plotly.express as px

vecs, labels = reduce_dimensions(w2v_model)

MAX_WORDS=200
fig = px.scatter(x=vecs[:MAX_WORDS,0], y=vecs[:MAX_WORDS,1], text=labels[:MAX_WORDS])
fig.show(renderer="colab") # esto para plotly en colab

In [ ]:
# Graficar los embedddings en 3D

vecs, labels = reduce_dimensions(w2v_model,3)

fig = px.scatter_3d(x=vecs[:MAX_WORDS,0], y=vecs[:MAX_WORDS,1], z=vecs[:MAX_WORDS,2],text=labels[:MAX_WORDS])
fig.update_traces(marker_size = 2)
fig.show(renderer="colab") # esto para plotly en colab

In [ ]:
# También se pueden guardar los vectores y labels como tsv para graficar en
# http://projector.tensorflow.org/


vectors = np.asarray(w2v_model.wv.vectors)
labels = list(w2v_model.wv.index_to_key)

np.savetxt("vectors.tsv", vectors, delimiter="\t")

with open("labels.tsv", "w") as fp:
    for item in labels:
        fp.write("%s\n" % item)

### Alumno

- Crear sus propios vectores con Gensim basado en lo visto en clase con otro dataset.
- Probar términos de interés y explicar similitudes en el espacio de embeddings (sacar conclusiones entre palabras similitudes y diferencias).
- Graficarlos.
- Obtener conclusiones.